QMzyme
==============================


![](logo.png)


### Jump to Contents
- #### [Installation](#Installation)
- #### [Dependencies](#Dependencies)
- #### [Getting Started](#Getting-Started)
    1. [Generate QMzyme object](#Step-1.-Generate-your-QMzyme-object)
    2. [Define catalytic center](#Step-2.-Define-the-catalytic-center)
    3. [Select subsystem](#Step-3.-Select-the-subsystem)
    4. [Truncate subsystem](#Step-4.-Truncate-the-subsystem)
    5. [Resulting model](#Step-5.-Resulting-model)

## Installation
```
pip install QMzyme
```

## Dependencies
```
- rdkit
- numpy
- aqme
- (optional) MDAnalysis
```

## Getting Started

In [8]:
import QMzyme as qmz
from importlib_resources import files, as_file
import inspect
from IPython.display import HTML, display

In [ ]:
def set_background(color):    
    script = (
        "var cell = this.closest('.jp-CodeCell');"
        "var editor = cell.querySelector('.jp-Editor');"
        "editor.style.background='{}';"
        "this.parentNode.removeChild(this)"
    ).format(color)
    
    display(HTML('<img src onerror="{}" style="display:none">'.format(script)))

## Step 1. Generate your QMzyme object

#### What arguments does QMzyme.GenerateModel take?

In [9]:
set_background('honeydew')
print(inspect.signature(qmz.GenerateModel))

(calculation='QM-only', protein_file=None, pdb_code=None, save_json=True, verbose=True)


#### Initialize model from PDB file:

In [3]:
file = str(files('QMzyme.data').joinpath('1oh0_equ_from_amber_sim.pdb'))
model = qmz.GenerateModel(protein_file=file)

#By default information is printed out after each function call. To turn this off for all calls 
# pass verbose=False to GenerateModel(). Otherwise, you can set verbose True or False in each of the 
# functions individually.

INITIALIZING... QMZYME OBJECT: 1oh0_equ_from_amber_sim
TIMESTAMP: 2024-02-13 09:48:27



## Step 2. Define the catalytic center

#### What arguments does QMzyme.GenerateModel.catalytic_center take?

In [4]:
print(inspect.signature(qmz.GenerateModel.catalytic_center))

(self, sel='', res_name=None, res_number=None, chain=None, output_file=None, save_file=True, save_json=None, verbose=None)


#### You can specify certain residue identifiers like the residue name and/or residue number. If you use a non-unique specifier (like res_name='ALA'), every Alanine residue will be selected, and a warning will pop up to ensure this was your intention. 

In [5]:
model.catalytic_center(res_number=263, save_file=False)

#I set save_file=False because I don't need the PDB file of the catalytic center,
# but you might want to visualize that to ensure it's what you intended.

INITIALIZING... CATALYTIC CENTER
TIMESTAMP: 2024-02-13 09:48:36.738592
DEFINITION: 
N_ATOMS: 37
The following object attributes are now available:
	self.catalytic_center_definition
	self.catalytic_center_mol
	self.catalytic_center_pdb


#### If you have MDAnalysis installed, you can use the sel argument to pass a string selection command that MDAnalysis will parse:

In [6]:
model.catalytic_center(sel='resid 263', save_json=False, save_file=False)

#I set save_json=False here because an error will pop up if we have 
# the same model object and try to write multiple catalytic centers 
# to its corresponding json file. If you generate a new model object
# in the same directory it will create a new json file and append a
# number identifier to it. 

INITIALIZING... CATALYTIC CENTER
TIMESTAMP: 2024-02-13 09:49:00.210642
DEFINITION: 
N_ATOMS: 37
The following object attributes are now available:
	self.catalytic_center_definition
	self.catalytic_center_mol
	self.catalytic_center_pdb


## Step 3. Select the subsystem

#### You must define some cutoff distance in Å. Any residue with at least one atom within this distance of at least one atom of the catalytic center will be included in the subsystem.

In [7]:
model.subsystem(distance_cutoff=5, save_file=False)

INITIALIZING... SUBSYSTEM SELECTION
TIMESTAMP: 2024-02-13 09:49:14.307705
CUTOFF: 5
OUTPUT_FILE: 1oh0_equ_from_amber_sim_subsystem_distance_cutoff5.pdb
N_ATOMS: 427
The following object attributes have been generated:
	self.distance_cutoff
	self.subsystem_mol
	self.subsystem_pdb



## Step 4. Truncate the subsystem

#### We will do this following the default truncation scheme of cutting only at backbone atoms of terminal residues (i.e., if the subsystem is comprised of residues 1, 2, 3 and 5, the N termini of residues 1 and 5 will be cut and the C termini of residues 3 and 5 will be cut.

In [ ]:
model.truncate(save_file=False)

## Step 5. Resulting model

#### As the code was running, a json file was created each step of the way to record information about the creation process and the resulting model. Creation of this json file can be turned off for all steps by setting save_json=False in qmz.GenerateModel(), or at any of the individual steps. Let's take a look at what information is stored in the json:

In [ ]:
import json

In [ ]:
with open(f'{model.protein_prefix}_QMzyme.json') as j:
    model_info = json.load(j)

In [ ]:
model_info["Starting structure"]

In [ ]:
model_info["Catalytic center"]

##### You will see there is a section "QMzyme 1" with information on how the subsystem was selected and what residues comprise the truncated subsystem. If you were to run model.subsystem() again, it would create a new entry under "QMyme 2". 

In [ ]:
model_info["QMzyme 1"]["Subsystem selection"]

In [ ]:
model_info["QMzyme 1"]["Truncated subsystem"]

#### Copyright
Copyright (c) 2023, Heidi Klem

#### Acknowledgements
Project architecture initiated through the 
[Computational Molecular Science Python Cookiecutter](https://github.com/molssi/cookiecutter-cms).